# VQE-optimized initial states for KQD
This is an attempt to integrate the Variational Quantum Eigensolver (VQE) and the Krylov Quantum Diagonalization (KQD) methods in order to get better approximations of the minimization problems that comes up in various domains. It is inspired from how the integration of the Sample-based quantum diagonalization and KQD methods gave the much more efficient Sample-based Krylov Quantum Diagonalization (SKQD) method, as discussed in the IBM Learning Platform [1]. 

In this work, we are obtaining an estimate of the initial state for the Krylov diagonalization by using the VQE method. The example being considered is the classic example of the Tranverse Field Ising Model (TFIM), with the Hamiltonian

$$H = -J \sum_{i} {Z_i Z_{i+1}} - h \sum_{i} {X_i},$$

where $J$ is the coupling strength between neighbouring spins, and $h$ is the strength of the transverse magnetic field [2].

## The VQE optimization part:

In [1]:
# we will first perform the necessary imports:
import numpy as np
import math
import sys

np.set_printoptions(threshold=5000)
#np.set_printoptions(threshold=np.inf)
np.set_printoptions(linewidth=np.inf)

import itertools
from qiskit.transpiler import CouplingMap

from qiskit.quantum_info import SparsePauliOp
from qiskit.quantum_info import Pauli


In [ ]:
# Define the problem Hamiltonian.

# TFIM Hamiltonian:
n_qubits = 10
# coupling strength and magnetic field
h_i = -1
JZ = -1
 
# Define the Hamiltonian:
H_int = [["I"] * n_qubits for _ in range(2 * (n_qubits - 1)+1)]
for i in range(n_qubits - 1):
    H_int[i][i] = "Z"
    H_int[i][i + 1] = "Z"
for i in range(n_qubits - 1, 2*(n_qubits - 1)+1):
    H_int[i][i - n_qubits] = "X"

H_int = ["".join(term) for term in H_int]
H_tot = [
    (term, -JZ)
    if term.count("Z") == 2
    else (term, -h_i)
    for term in H_int
]
 
# Get the Hamiltonian operator
H_op = SparsePauliOp.from_list(H_tot)

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.transpiler import PassManager
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
#'''
# Option 1: tilted ansatz construction:
ansatz = QuantumCircuit(n_qubits)
from qiskit.circuit import ParameterVector
p_vector = ParameterVector('p', n_qubits)
[ansatz.ry(p_vector[i], i) for i in range(n_qubits)]
#'''
'''
# Option 2: N_Local/efficient_su2 ansatz construction:
from qiskit.circuit.library import efficient_su2, n_local
ansatz = n_local(n_qubits, 'ry', 'cx', 'linear', reps=1)
#ansatz = efficient_su2(n_qubits, su2_gates=["rx"], entanglement="linear", reps=1)
'''
# initial set of parameters for the optimization:
#np.random.seed(42)
#x0 = 2 * np.pi * np.random.random(ansatz.num_parameters)
x0 = 2 * np.pi * np.zeros(ansatz.num_parameters)


In [ ]:
# SciPy minimizer routine
from scipy.optimize import minimize

# Plotting functions

print(f'Decomposed ansatz depth: {ansatz.decompose(reps=3).depth()}')
#ansatz.decompose(reps = 2).draw("mpl")
#ansatz.draw('mpl', style={'fontsize': 30, 'subfontsize': 23})

from qiskit_ibm_runtime import QiskitRuntimeService
# We will start by using a local simulator
from qiskit_aer import AerSimulator
backend = AerSimulator()

target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)

# Use the pass manager and draw the resulting circuit
ansatz_isa = pm.run(ansatz)
#ansatz_isa.draw(output="mpl", idle_wires=False, style="iqp")

hamiltonian_isa = H_op.apply_layout(ansatz_isa.layout)

# Import an estimator, this time from qiskit (we will import from Runtime for real hardware)
from qiskit.primitives import BackendEstimatorV2

# generate a simulator that mimics the real quantum system
#backend_sim = AerSimulator.from_backend(backend)
#estimator = BackendEstimatorV2(backend=backend_sim) 

estimator = BackendEstimatorV2(backend=backend)

def cost_func(params, ansatz, H, estimator):
    pub = (ansatz, [H], [params])
    result = estimator.run(pubs=[pub]).result()
    energy = result[0].data.evs[0]
    
    cost_history_dict["iters"] += 1
    cost_history_dict["prev_vector"] = params
    cost_history_dict["cost_history"].append(energy)
    print(f"Iters. done: {cost_history_dict['iters']} [Current cost: {energy}]")

    return energy

cost_history_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
}

res = minimize(
    cost_func,
    x0,
    args=(ansatz_isa, hamiltonian_isa, estimator),
    method="COBYLA",
    #optimizer = COBYLA,
    tol=1e-6,
    options={"maxiter": 1000, "disp": True},
)

print(getattr(res, "fun"))
print(res)


In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.plot(range(cost_history_dict["iters"]), cost_history_dict["cost_history"])
ax.set_xlabel("Iterations",fontsize=15, weight='bold')
ax.set_ylabel("Cost",fontsize=15, weight='bold')
plt.xticks(fontsize=15, weight='bold')
plt.yticks(fontsize=15, weight='bold')
plt.show()

In [ ]:
# creating the circuit that can serve as the initial state for the Krylov method, using the optimized values of the parameters:
optimal_params = res.x
optimal_circuit = ansatz_isa.assign_parameters(optimal_params)
optimal_circuit.draw('mpl')

In [ ]:
# to find the bitstring corresponding to the optimal parameter values, and map this to the KQD circuit as the initial state:

optimal_circuit.measure_all()
##optimal_circuit.draw('mpl')

from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit.visualization import plot_histogram

sampler = Sampler(mode=backend)
job = sampler.run([(optimal_circuit)], shots=100_000)
counts = job.result()[0].data.meas.get_counts()
sorted_counts_desc = sorted(counts.items(), key=lambda item: item[1], reverse=True)
print(sorted_counts_desc)
plot_histogram(counts, sort='value_desc')

max_val_string = max(counts, key = counts.get)
print(f'Estimated ground state: {max_val_string}')


## The KQD method:

The KQD part has been adapted from the IBM Learning Platform.

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pylab as plt
from typing import Union, List
import warnings

np.set_printoptions(threshold=5000)
#np.set_printoptions(threshold=np.inf)
np.set_printoptions(linewidth=np.inf)

from qiskit.quantum_info import SparsePauliOp, Pauli
from qiskit.circuit import Parameter
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.synthesis import LieTrotter
 
# from qiskit.providers.fake_provider import Fake20QV1
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator, Batch
 
import itertools as it
 
warnings.filterwarnings("ignore")

In [ ]:
def solve_regularized_gen_eig(
    h: np.ndarray,
    s: np.ndarray,
    threshold: float,
    k: int = 1,
    return_dimn: bool = False,
) -> Union[float, List[float]]:
    """
    Method for solving the generalized eigenvalue problem with regularization
 
    Args:
        h (numpy.ndarray):
            The effective representation of the matrix in our Krylov subspace
        s (numpy.ndarray):
            The matrix of overlaps between vectors of our Krylov subspace
        threshold (float):
            Cut-off value for the eigenvalue of s
        k (int):
            Number of eigenvalues to return
        return_dimn (bool):
            Whether to return the size of the regularized subspace
 
    Returns:
        lowest k-eigenvalue(s) that are the solution of the regularized generalized eigenvalue problem
 
 
    """
    s_vals, s_vecs = sp.linalg.eigh(s)
    s_vecs = s_vecs.T
    good_vecs = np.array([vec for val, vec in zip(s_vals, s_vecs) if val > threshold])
    h_reg = good_vecs.conj() @ h @ good_vecs.T
    s_reg = good_vecs.conj() @ s @ good_vecs.T
    if k == 1:
        if return_dimn:
            return sp.linalg.eigh(h_reg, s_reg)[0][0], len(good_vecs)
        else:
            return sp.linalg.eigh(h_reg, s_reg)[0][0]
    else:
        if return_dimn:
            return sp.linalg.eigh(h_reg, s_reg)[0][:k], len(good_vecs)
        else:
            return sp.linalg.eigh(h_reg, s_reg)[0][:k]

In [ ]:
# Get Hamiltonian restricted to fixed-particle states
fixed_particle_H = np.zeros((n_qubits, n_qubits))
for i in range(n_qubits):
    for j in range(i + 1):
        for p, coeff in H_op.to_list():
            p_x = Pauli(p).x
            p_z = Pauli(p).z
            #print(f'P_x: {p_x}')
            #print(f'P_z: {p_z}')
            if all(p_x[k] == ((i == k) + (j == k)) % 2 for k in range(n_qubits)):
                sgn = ((-1j) ** sum(p_z[k] and p_x[k] for k in range(n_qubits))) * (
                    (-1) ** p_z[i]
                )
            else:
                sgn = 0
            fixed_particle_H[i, j] += sgn * coeff
for i in range(n_qubits):
    for j in range(i + 1, n_qubits):
        fixed_particle_H[i, j] = np.conj(fixed_particle_H[j, i])
 
# Set dt according to spectral norm
dt = np.pi / np.linalg.norm(fixed_particle_H, ord=2)

# Set parameters for quantum Krylov algorithm
krylov_dim = 8  # size of krylov subspace
num_trotter_steps = 10
dt_circ = dt / num_trotter_steps


In [ ]:
# set the VQE optimal ansatz circuit as the State Preparation circuit for KQD:
VQE_state = max_val_string
qc_state_prep = QuantumCircuit(n_qubits)
[qc_state_prep.x(i) for i in range(len(VQE_state)) if VQE_state[i]=='1']
qc_state_prep.draw('mpl')

In [ ]:
#'''
# Modified Trotter-step circuit:
t = Parameter("t")
# Create instruction for rotation about X+ZZ:
Rxyz_circ = QuantumCircuit(2)

#Rxyz_circ.rzz(2 * t, 0, 1)
Rxyz_circ.rzz(2 * JZ * t, 0, 1)
Rxyz_instr = Rxyz_circ.to_instruction(label="RZZ")

interaction_list = [
    [[i, i + 1] for i in range(0, n_qubits - 1, 2)],
    [[i, i + 1] for i in range(1, n_qubits - 1, 2)],
]  # linear chain

#rx_qubit_list = 
qr = QuantumRegister(n_qubits)
trotter_step_circ = QuantumCircuit(qr)
for i, color in enumerate(interaction_list):
    for interaction in color:
        if interaction[0]%2==0:
            #trotter_step_circ.rx(2*t, interaction[0])
            #trotter_step_circ.rx(2*t, interaction[1])
            trotter_step_circ.rx(2*h_i*t, interaction[0])
            trotter_step_circ.rx(2*h_i*t, interaction[1])
            
        trotter_step_circ.append(Rxyz_instr, interaction)
        
    if i < len(interaction_list) - 1:
        trotter_step_circ.barrier()
    
reverse_trotter_step_circ = trotter_step_circ.reverse_ops()
#'''

In [ ]:
'''
from qiskit.circuit import QuantumRegister
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.synthesis import LieTrotter

t = Parameter("t")

evol_gate = PauliEvolutionGate(
    H_op, time=t, synthesis=LieTrotter(reps=num_trotter_steps)
)  # `U` operator
 
qr = QuantumRegister(n_qubits)
trotter_step_circ = QuantumCircuit(qr)
trotter_step_circ.append(evol_gate, qargs=qr)

reverse_trotter_step_circ = trotter_step_circ.reverse_ops()
'''

In [ ]:
qc_evol = QuantumCircuit(qr)
for step in range(num_trotter_steps):
    if step % 2 == 0:
        qc_evol = qc_evol.compose(trotter_step_circ)
    else:
        qc_evol = qc_evol.compose(reverse_trotter_step_circ)
 
qc_evol.decompose().draw("mpl", fold=-1, scale=0.5)

In [ ]:
# Create controlled version of the state_prep circuit:
controlled_state_prep = qc_state_prep.to_gate().control(1)

# Parameters for the template circuits
parameters = []
for idx in range(1, krylov_dim):
    parameters.append(dt_circ * (idx))
    
# Create modified Hadamard test circuit
qr = QuantumRegister(n_qubits + 1)
qc = QuantumCircuit(qr)
qc.h(0)
qc.compose(controlled_state_prep, list(range(n_qubits + 1)), inplace=True)
qc.barrier()
qc.compose(qc_evol, list(range(1, n_qubits + 1)), inplace=True)
qc.barrier()
qc.x(0)
qc.compose(controlled_state_prep.inverse(), list(range(n_qubits + 1)), inplace=True)
qc.x(0)
qc.decompose().draw("mpl", fold=-1)

print(
    "The optimized circuit has 2Q gates depth: ",
    qc.decompose().decompose().depth(lambda x: x[0].num_qubits == 2),
)

In [ ]:
target = backend.target
basis_gates = list(target.operation_names)
pm = generate_preset_pass_manager(
    optimization_level=3, backend=backend, basis_gates=basis_gates
)
 
qc_trans = pm.run(qc)

print(qc_trans.depth(lambda x: x[0].num_qubits == 2))
print(qc_trans.count_ops())
qc_trans.draw("mpl", fold=-1, idle_wires=False, scale=0.5)


In [ ]:
# Define observables to measure for S
observable_S_real = "I" * (n_qubits) + "X"
observable_S_imag = "I" * (n_qubits) + "Y"
 
observable_op_real = SparsePauliOp(
    observable_S_real
)  # define a sparse pauli operator for the observable
observable_op_imag = SparsePauliOp(observable_S_imag)
 
layout = qc_trans.layout  # get layout of transpiled circuit
observable_op_real = observable_op_real.apply_layout(
    layout
)  # apply physical layout to the observable
observable_op_imag = observable_op_imag.apply_layout(layout)
observable_S_real = (
    observable_op_real.paulis.to_labels()
)  # get the label of the physical observable
observable_S_imag = observable_op_imag.paulis.to_labels()
 
observables_S = [[observable_S_real], [observable_S_imag]]
 
 
# Define observables to measure for H
# Hamiltonian terms to measure
observable_list = []
for pauli, coeff in zip(H_op.paulis, H_op.coeffs):
    # print(pauli)
    observable_H_real = pauli[::-1].to_label() + "X"
    observable_H_imag = pauli[::-1].to_label() + "Y"
    observable_list.append([observable_H_real])
    observable_list.append([observable_H_imag])
    

In [ ]:
layout = qc_trans.layout
 
observable_trans_list = []
for observable in observable_list:
    observable_op = SparsePauliOp(observable)
    observable_op = observable_op.apply_layout(layout)
    observable_trans_list.append([observable_op.paulis.to_labels()])
 
observables_H = observable_trans_list
 
 
# Define a sweep over parameter values
params = np.vstack(parameters).T

 
# Estimate the expectation value for all combinations of
# observables and parameter values, where the pub result will have
# shape (# observables, # parameter values).
pub = (qc_trans, observables_S + observables_H, params)

from qiskit.quantum_info import StabilizerState, Pauli

qc_cliff = qc.assign_parameters({t: 0})
 
# Get expectation values from experiment
S_expval_real = StabilizerState(qc_cliff).expectation_value(
    Pauli("I" * (n_qubits) + "X")
)
S_expval_imag = StabilizerState(qc_cliff).expectation_value(
    Pauli("I" * (n_qubits) + "Y")
)
 
# Get expectation values
S_expval = S_expval_real + 1j * S_expval_imag
 
H_expval = 0
for obs_idx, (pauli, coeff) in enumerate(zip(H_op.paulis, H_op.coeffs)):
    # Get expectation values from experiment
    expval_real = StabilizerState(qc_cliff).expectation_value(
        Pauli(pauli[::-1].to_label() + "X")
    )
    expval_imag = StabilizerState(qc_cliff).expectation_value(
        Pauli(pauli[::-1].to_label() + "Y")
    )
    expval = expval_real + 1j * expval_imag
 
    # Fill-in matrix elements
    H_expval += coeff * expval
 
 
print(H_expval)


In [ ]:
with Batch(backend=backend) as batch:
    # Estimator
    #estimator = Estimator(mode=batch, options=options)
    estimator = Estimator(mode=batch)
 
    job = estimator.run([pub], precision=0.006)

# Store the outputs as 'results'.
results = job.result()[0]

prefactors = [
    np.exp(-1j * sum([c for p, c in H_op.to_list() if "Z" in p]) * i * dt)
    for i in range(1, krylov_dim)
]

# Assemble S, the overlap matrix of dimension D:
S_first_row = np.zeros(krylov_dim, dtype=complex)
S_first_row[0] = 1 + 0j
 
# Add in ancilla-only measurements:
for i in range(krylov_dim - 1):
    # Get expectation values from experiment
    expval_real = results.data.evs[0][0][i]  # automatic extrapolated evs if ZNE is used
    expval_imag = results.data.evs[1][0][i]  # automatic extrapolated evs if ZNE is used
 
    # Get expectation values
    expval = expval_real + 1j * expval_imag
    S_first_row[i + 1] += prefactors[i] * expval
 
S_first_row_list = S_first_row.tolist()  # for saving purposes
 
 
S_circ = np.zeros((krylov_dim, krylov_dim), dtype=complex)
 
# Distribute entries from first row across matrix:
for i, j in it.product(range(krylov_dim), repeat=2):
    if i >= j:
        S_circ[j, i] = S_first_row[i - j]
    else:
        S_circ[j, i] = np.conj(S_first_row[j - i])

#from sympy import Matrix
#Matrix(S_circ)

In [ ]:
import itertools
 
# Assemble S, the overlap matrix of dimension D:
H_first_row = np.zeros(krylov_dim, dtype=complex)
H_first_row[0] = H_expval
 
for obs_idx, (pauli, coeff) in enumerate(zip(H_op.paulis, H_op.coeffs)):
    # Add in ancilla-only measurements:
    for i in range(krylov_dim - 1):
        # Get expectation values from experiment
        expval_real = results.data.evs[2 + 2 * obs_idx][0][
            i
        ]  # automatic extrapolated evs if ZNE is used
        expval_imag = results.data.evs[2 + 2 * obs_idx + 1][0][
            i
        ]  # automatic extrapolated evs if ZNE is used
 
        # Get expectation values
        expval = expval_real + 1j * expval_imag
        H_first_row[i + 1] += prefactors[i] * coeff * expval
 
H_first_row_list = H_first_row.tolist()
 
H_eff_circ = np.zeros((krylov_dim, krylov_dim), dtype=complex)
 
# Distribute entries from first row across matrix:
for i, j in itertools.product(range(krylov_dim), repeat=2):
    if i >= j:
        H_eff_circ[j, i] = H_first_row[i - j]
    else:
        H_eff_circ[j, i] = np.conj(H_first_row[j - i])
 
#Matrix(H_eff_circ)

In [ ]:
gnd_en_circ_est_list = []
for d in range(1, krylov_dim + 1):
    # Solve generalized eigenvalue problem
    gnd_en_circ_est = solve_regularized_gen_eig(
        H_eff_circ[:d, :d], S_circ[:d, :d], threshold=1e-2
    )    # threshold changed from 1e-1 to 1e-2
    gnd_en_circ_est_list.append(gnd_en_circ_est)
    print("The estimated ground state energy is: ", gnd_en_circ_est)

#gs_en = fixed_particle_gs(H_op, n_qubits)
#gs_en = fixed_particle_gs(H_op)

gs_en = np.min(np.linalg.eig(H_op.to_matrix())[0])

print("The exact ground state energy is ", gs_en)


In [ ]:
plt.plot(
    range(1, krylov_dim + 1),
    gnd_en_circ_est_list,
    color="blue",
    linestyle="-.",
    label="KQD estimate",
)
plt.plot(
    range(1, krylov_dim + 1),
    [gs_en] * krylov_dim,
    color="red",
    linestyle="-",
    label="exact",
)
plt.xticks(range(1, krylov_dim + 1), range(1, krylov_dim + 1))
plt.legend()
plt.xlabel("Krylov space dimension", fontsize=12)
plt.ylabel("Energy", fontsize=12)
plt.title("Estimating target state's energy with Krylov Quantum Diagonalization", fontsize=12)
plt.show()


#### References:

1. IBM Quantum Platform. Sample-based Krylov Quantum Diagonalization (SKQD). 
   [https://quantum.cloud.ibm.com/learning/en/courses/quantum-diagonalization-algorithms/skqd](https://quantum.cloud.ibm.com/learning/en/courses/quantum-diagonalization-algorithms/skqd)

2. IBM Quantum Platform - Tutorials. Transverse-Field Ising Model with Q-CTRL's Performance Management.
   https://quantum.cloud.ibm.com/docs/en/tutorials/transverse-field-ising-model

*This code was part of the work done as part of the Qiskit Advocate Mentorship Programme (QAMP) 2025 project No.: 31.*\
*Mentors: Dr. Soham Pal, Dr. Shiplu Sarker,*\
*Mentees: Abdullah Afzal, Michael Papadopoulos, Gayathree M. Vinod.*\
*This notebook was prepared by Gayathree M. Vinod and verified by Michael Papadopoulos.*

Qiskit version used: 2.2.3